<a href="https://colab.research.google.com/github/AleAguirreM/Analitica3AA/blob/main/c_modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib

In [2]:
conn = sql.connect("/content/drive/MyDrive/db_moviesSQL")

In [3]:
curr = conn.cursor()

In [4]:
curr.execute("SELECT name FROM sqlite_master WHERE type = 'table'")

In [5]:
curr.fetchall()

[('ratings',),
 ('movies',),
 ('movies_sel',),
 ('movies_n',),
 ('ratings_final',),
 ('movies_final',),
 ('full_ratings',)]

In [6]:
pd.read_sql("select * from ratings", conn)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [ ]:
#try:
    # Intentar agregar la columna fecha_fecha si no existe
    #curr.execute("ALTER TABLE ratings ADD COLUMN fecha_fecha DATE")
#except sql.OperationalError as e:
    # Si la columna ya existe, mostrar un mensaje de advertencia
    #print("La columna 'fecha_fecha' ya existe en la tabla 'ratings'.")

# Actualizar la nueva columna con los valores convertidos de la columna existente
#curr.execute("UPDATE ratings SET fecha_fecha = DATE(timestamp)")


La columna 'fecha_fecha' ya existe en la tabla 'ratings'.


In [7]:
pd.read_sql("select * from movies", conn)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [8]:
######################################################################
################## 1. sistemas basados en popularidad ###############
#####################################################################


##### recomendaciones basado en popularidad ######

#### mejores calificadas que tengan calificacion
pd.read_sql("""select title,
            avg(rating) as avg_rat,
            count(*) as read_num
            from full_ratings
            where rating<>0
            group by title
            order by avg_rat desc
            limit 10
            """, conn)

,title,avg_rat,read_num
0,"Streetcar Named Desire, A (1951)",4.475000,20
1,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.333333,27
2,"Philadelphia Story, The (1940)",4.310345,29
3,Lawrence of Arabia (1962),4.300000,45
4,In the Name of the Father (1993),4.300000,25
5,Hoop Dreams (1994),4.293103,29
6,"Godfather, The (1972)",4.289062,192
7,Harold and Maude (1971),4.288462,26
8,Logan (2017),4.280000,25
9,Fight Club (1999),4.272936,218


In [10]:
#### los mas leidos con promedio de los que calficaron ###
pd.read_sql("""select title,
            avg(iif(rating = 0, Null, rating)) as avg_rat,
            count(*) as read_num
            from full_ratings
            group by title
            order by read_num desc
            """, conn)

,title,avg_rat,read_num
0,"Silence of the Lambs, The (1991)",4.161290,279
1,"Matrix, The (1999)",4.192446,278
2,Star Wars: Episode IV - A New Hope (1977),4.231076,251
3,Jurassic Park (1993),3.750000,238
4,Braveheart (1995),4.031646,237
...,...,...,...
1289,"Animatrix, The (2003)",3.700000,20
1290,Anastasia (1997),3.350000,20
1291,Adventures in Babysitting (1987),3.325000,20
1292,"6th Day, The (2000)",2.800000,20


In [11]:

#######################################################################
######## 2.1 Sistema de recomendación basado en contenido un solo producto - Manual ########
#######################################################################

movies=pd.read_sql('select * from movies_final', conn )

movies.info()
movies['year_pub']=movies.year_pub.astype('int')
movies.info()

##### escalar para que año esté en el mismo rango ###

sc=MinMaxScaler()
movies[["year_sc"]]=sc.fit_transform(movies[['year_pub']])



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1294 entries, 0 to 1293
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   movieId    1294 non-null   int64 
 1   title      1294 non-null   object
 2   genres     1294 non-null   object
 3   movieId:1  1294 non-null   int64 
 4   cnt_rat    1294 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 50.7+ KB


AttributeError: 'DataFrame' object has no attribute 'year_pub'

In [13]:
## eliminar filas que no se van a utilizar ###

movies_dum1=movies.drop(columns=['userId', 'movieId', 'ratings'])


KeyError: "['userId', 'ratings'] not found in axis"

In [ ]:
#### convertir a dummies

movies_dum1['gender'].nunique()
movies_dum1['publisher'].nunique()

col_dum=['book_author','publisher']
movies_dum2=pd.get_dummies(movies_dum1,columns=col_dum)
movies_dum2.shape

joblib.dump(movies_dum2,"salidas\\movies_dum2.joblib") ### para utilizar en segundos modelos
